<a href="https://colab.research.google.com/github/PTson2207/Classification-heartcsv/blob/main/Classification_heart_with_Kfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/content/drive/MyDrive/DataVIZ/DatViz/heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
from sklearn.model_selection import train_test_split

In [9]:
data = df
X = data.iloc[:, data.columns != 'target']
y = data.iloc[:, data.columns == 'target']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
print('Train dataset:', len(X_train))
print('Test dataset:', len(X_test))
print('Dataset:', len(X_train) + len(X_test))

Train dataset: 212
Test dataset: 91
Dataset: 303


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  KFold, cross_val_score
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report

In [7]:
def printing_Kfold_scores(x_train_data, y_train_data):

    fold = KFold(5, shuffle=False)

    # các thông số C khác nhau
    c_param_range = [0.1, 1, 10, 100, 10000] # tham số tự do

    results_table = pd.DataFrame(index=range(len(c_param_range), 2), columns=['C_parameter', 'Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # xuất 2 thông tin của Kfold
    j = 0 
    for c_param in c_param_range:
        print('-----------------------------')
        print('Tham số C:', c_param)
        print('-----------------------------')
        print('')

        recall_accs = []
        for iteration, indices in enumerate(fold.split(x_train_data)): 
            lr = LogisticRegression(C = c_param, penalty = 'l1', solver='liblinear')   
            lr.fit(x_train_data.iloc[indices[0],:],y_train_data.iloc[indices[0],:].values.ravel())  
            y_pred_undersample = lr.predict(x_train_data.iloc[indices[1],:].values)  
            recall_acc = recall_score(y_train_data.iloc[indices[1],:].values,y_pred_undersample) 
            recall_accs.append(recall_acc)
            print('Vòng lặp ', iteration,': Điểm recall = ', recall_acc)

        # tính giá trị trung bình của các thông số ở trên
        results_table.loc[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Trung bình điểm recall:', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].astype(float).idxmax()]['C_parameter']

    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Model tốt nhất từ cross-validation với tham số C = ', best_c)
    print('*********************************************************************************')

    return best_c

In [13]:
best_c = printing_Kfold_scores(X_train, y_train)

-----------------------------
Tham số C: 0.1
-----------------------------

Vòng lặp  0 : Điểm recall =  0.7916666666666666
Vòng lặp  1 : Điểm recall =  0.8947368421052632
Vòng lặp  2 : Điểm recall =  0.8695652173913043
Vòng lặp  3 : Điểm recall =  0.8846153846153846
Vòng lặp  4 : Điểm recall =  0.7619047619047619

Trung bình điểm recall: 0.8404977745366761

-----------------------------
Tham số C: 1
-----------------------------

Vòng lặp  0 : Điểm recall =  0.9166666666666666
Vòng lặp  1 : Điểm recall =  0.9473684210526315
Vòng lặp  2 : Điểm recall =  0.9130434782608695
Vòng lặp  3 : Điểm recall =  0.8846153846153846
Vòng lặp  4 : Điểm recall =  0.8095238095238095

Trung bình điểm recall: 0.8942435520238723

-----------------------------
Tham số C: 10
-----------------------------

Vòng lặp  0 : Điểm recall =  0.8333333333333334
Vòng lặp  1 : Điểm recall =  0.8947368421052632
Vòng lặp  2 : Điểm recall =  0.9130434782608695
Vòng lặp  3 : Điểm recall =  0.8076923076923077
Vòng lặp  4 :